# 项目P3：整理 OpenStreetMap 数据

## 地图数据

[OpenStreetMap](https://www.openstreetmap.org/) 是一个开放的地图数据集，该数据包含了节点（nodes）、途径（ways）和相互关系（relations）这些主要信息，详情可参见其[维基页面](https://wiki.openstreetmap.org/wiki/OSM_XML)。


本项目选取了[北京地区](https://www.openstreetmap.org/relation/912940#map=8/40.255/116.463)的地图数据，从 [Mapzen](https://mapzen.com/data/metro-extracts/metro/beijing_china/)中直接下载而来。之所以选北京地区，是因为其数据量较其他城市更丰富，而且更为大家熟悉。

这篇报告是在 Jupyter Notebook 中完成的。首先让我们导入相应的工具包，其中 osm2csv.py 是自定义的 Python  代码模块，它将 osm 格式的地图数据转换成 csv 格式，这部分代码来自于 Udacity 课程中的练习案例。


In [1]:
import pandas as pd
import re
import osm2csv

然后我们使用 osm2csv.py 中的 `process_map()` 函数将北京的地图数据进行处理，转换成一系列csv格式的数据。

In [2]:
osm2csv.process_map('beijing_china.osm', validate=False)  

我们使用Linux中的命令来查看现有的数据集。其中 beijing_china.osm 是原始数据，数据量有175M。其他以 .csv 结尾的文件是新生成的数据，nodes.csv 和 nodes_tags.csv 是关于节点的数据文件；ways.csv、ways_nodes.csv 和 ways_tags.csv 是关于途径的数据文件。

In [3]:
!du -h *.csv *.osm

 65M	nodes.csv
2.9M	nodes_tags.csv
6.9M	ways.csv
 23M	ways_nodes.csv
8.0M	ways_tags.csv
175M	beijing_china.osm
 18M	sample_beijing_china.osm


数据中的tag属性描述了关于节点和途径的具体信息，是我们接下来要关注的重点，所以这里主要处理 nodes_tags.csv 和 ways_tags.csv 这两份数据文件，并将它们存储到 DataFrame 中。

In [4]:
node_tag = pd.read_csv('nodes_tags.csv')
way_tag = pd.read_csv('ways_tags.csv')

## 地图数据中的问题

在观察数据以后，有以下几类数据问题需要处理：
* 电话号码格式混乱（比如 +86 10 6582 2892， (010)64629112，+86-10-60712288）。
* 存在错误的邮政编码（比如位数不对或者不在北京地区范围内）。
* 营业时间格式混乱不统一（比如 10am - 11pm、9:00 - 22:00、24小时、24h等）。
* 门牌号码错误（比如门牌号中没有包含数字）。
* 缺失name属性值（同一节点或途径，如果有zh（中文名）属性却没有name属性，则需要用zh的值进行补全）。

下面我们将会逐一处理这些问题，为了表述的简洁性，这里将数据清洗的代码块存储到 clean.py 文件中，在 notebook 中只需导入clean模块，并调用相应的处理函数即可。

In [5]:
import clean

### 清洗电话号码

下方列出了若干电话号码数据，观察其 value 值发现它们的格式不尽相同，所以需要将他们的格式进行统一。

In [6]:
node_tag[node_tag.key=='phone'].head()

,id,key,value,type
5510,600243400,phone,+86 10 6582 2892,regular
5560,600265656,phone,(010)64629112,regular
7709,984102277,phone,01051696505,regular
9909,1249791909,phone,+86-10-60712288,regular
14371,1422005598,phone,68716285;62555813,regular


这里我们规定统一的电话格式为：“国家编号（+86） + [区号（10）] + 号码”，对于手机号、400电话，则不需要使用区号。以下列出了 clean.py 代码中关于手机号码清洗的主要函数，分别对固定电话、手机电话和400电话进行了格式统一。

In [7]:
def style_phone_number(phone_value):
    '''将不符合标准格式的电话号码转换成标准格式。
    标准格式定义成：国家编号 + [区号] + 号码。'''

    # 去除非数字的字符
    digit_value = ''
    for char in phone_value:
        if char.isdigit():
            digit_value += char
    
    # 定义一些可能出现的号码格式
    # 以下列表中每一个元素都是数据中出现的号码格式，元组的第一个元素代表打头的数字，第二个元素代表位数
    phone_style = [('8610',12), ('86010', 13), ('008610', 14), ('010', 11), 
                    ('10', 10), ('86', 10), ('', 8) ]
    mobile_style = [('86', 13), ('0086', 15), ('', 11)]
    special_style = [('86400', 12), ('400', 10)]
        
    # 按固定电话、移动电话、400电话三种形式分别进行电话号码格式的标准化    
    if (digit_value[:-8], len(digit_value)) in phone_style:
        styled_value = '+86 10 ' + digit_value[-8:]
    elif ((digit_value[:-11], len(digit_value)) in mobile_style) \
            and is_mobile_phone(digit_value[-11:]):
        styled_value = '+86 ' + digit_value[-11:]
    elif (digit_value[:-7], len(digit_value)) in special_style:
        styled_value = '+86 ' + digit_value[-10:]
    else: 
        styled_value = digit_value

    return styled_value


运行`clean.process_phone()`函数，用于清洗 node_tag 数据中的电话号码，其返回的结果是被丢弃的不正确的电话号码，除此之外其他号码都被标准格式替代。

In [8]:
clean.process_phone(node_tag)

['010670666611',
 '03126550752',
 '861013717888828',
 '0106765514',
 '86861064376299']

为了检查清洗的效果，我们再次重新查看之前给出的电话号码时，发现他们都被统一成了标准格式。

In [9]:
node_tag[node_tag.key=='phone'].head()

,id,key,value,type
5510,600243400,phone,+86 10 65822892,regular
5560,600265656,phone,+86 10 64629112,regular
7709,984102277,phone,+86 10 51696505,regular
9909,1249791909,phone,+86 10 60712288,regular
14371,1422005598,phone,+86 10 68716285;+86 10 62555813,regular


我们发现途径数据中也存在部分电话信息，所以也需要对 way_tag 数据中的电话号码进行同样的操作。返回结果为空，表示所有的号码都被标准化了，且没有无法被标准化的数据。

In [10]:
clean.process_phone(way_tag)

[]

### 清洗邮政编码

北京的邮编都是由6位数字组成，并且以100、101、102开头。我们需要将不符合这一条件的数据找出来，并将这些错误的邮编从数据中删除。

首先来观察若干邮政编码的值，下方列出的都是符合标准的邮编。

In [11]:
node_tag[node_tag.key=='postcode'].head()

,id,key,value,type
5568,600265656,postcode,100027,addr
7497,973340625,postcode,100875,addr
7716,984102277,postcode,100061,addr
9292,1104307346,postcode,100101,addr
11156,1327095475,postcode,100015,addr


也有不符合标准的邮编在数据中出现，比如下面这条数据，不满足6位数字。

In [12]:
node_tag.loc[[64506]]

,id,key,value,type
64506,3470737818,postcode,3208,addr


以下是 clean.py 清洗代码中关于邮编的处理函数，我们将不符合北京地区邮编格式的数据删除。

In [13]:
def process_postcode(df):
    '''清洗数据中的邮政编码。
    返回值是被丢弃的错误的邮编。 '''

    wrong_list = []
    # 定义邮编的正则表达式，北京地区邮编以100、101、102开头，共6位数字
    postcode_parttern = r'^10[0-2]\d{3}$'
    
    # 判断邮编是否符合以上定义的正则表达式，不符合则从数据中删除
    for index, row in df[df.key=='postcode'].iterrows():
        code = row['value']
        if not re.fullmatch(postcode_parttern, code):
            wrong_list.append(code)
            df.drop(index, inplace=True)
            
    return wrong_list  

使用 `clean.process_postcode()` 函数分别对 node_tag 和 way_tag 数据进行邮编清洗，返回的结果是不符合标准的邮编，它们会被删除。

In [14]:
clean.process_postcode(node_tag)

['3208', '110101', '110023', '053600']

In [15]:
clean.process_postcode(way_tag)

['010-62332281', '10080', '10040', '10043']

### 清洗营业时间数据

首先还是来观察营业时间相应的数据。

In [16]:
node_tag[node_tag.key=='opening_hours'].head()

,id,key,value,type
1323,269700902,opening_hours,Mo-Su 09:00-17:00,regular
1995,290599918,opening_hours,Mo-Su 05:00-24:00,regular
5498,600238274,opening_hours,24/7,regular
5518,600243400,opening_hours,11:00-22:00,regular
7266,854692925,opening_hours,24/7,regular


营业时间数据相对比较杂乱，这里根据可获得时间信息的丰富程度，将营业时间统一成以月份、星期、小时排列的形式，具体参见 `clean.is_hour()` 函数中给出的十种时间格式。该函数用于判断营业时间是否符合我们定义的格式，如果不符合，需要对时间进行格式统一。由于时间数据的情况比较复杂多样，具体清洗的代码可参见 clean.py 文件中的营业时间清洗部分。

In [17]:
def is_hour(value): 
    '''判断营业时间是否满足统一的格式'''

    # 关于小时、星期、月份的正则表达式
    h = '\d{1,2}:\d{1,2}'  
    w = '(Mo|Tu|We|Th|Fr|Sa|Su)'
    m = '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
    md = m +' \d{1,2}'
    
    # 定义了营业时间的统一格式，共有10中形式
    p0 = '24/7'                    # 表示7天24小时都营业
    p1 = h + '-' + h               # e.g. 06:00-23:00
    p2 = w + '-' + w + ' ' + p1    # e.g. Mo-Su 06:00-23:00
    p3 = w + ' ' + p1              # e.g. Sat 09:30-22:00
    p4 = m + '-' + m + ' ' + p2    # e.g. Apr-Oct Mo-Su 05:00-24:00
    p5 = md + '-' + md + ' ' + p1  # e.g. Apr 1-Oct 31 05:00-24:00
    p6 = p2 + ', ' + p1            # e.g. Su-Fr 08:30-11:30, 13:30-17:00
    p7 = p1 + ', ' + p1            # e.g. 08:30-11:30, 13:30-17:00
    p8 = m + '-' + m + ' ' + p1    # e.g. Apr-Oct 08:00-17:00
    p9 = p1 + ',' + p1

    # 判断数据是否满足以上给出的统一格式，是则返回True，否则返回False
    if  (re.fullmatch(p1, value) \
        or re.fullmatch(p2, value) \
        or re.fullmatch(p3, value) \
        or re.fullmatch(p4, value) \
        or re.fullmatch(p5, value) \
        or re.fullmatch(p6, value) \
        or re.fullmatch(p7, value) \
        or re.fullmatch(p8, value) \
        or re.fullmatch(p9, value) \
        or value == p0):
        return True
    else:
        return False

这里先使用 `clean.find_mess_hour()` 函数找出 node_tag 数据中不符合要求的时间数据。

In [18]:
wrong_index, wrong_hour = clean.find_mess_hour(node_tag)
for h in wrong_hour:
    print(h)

Mo-Fr 10:00-22:00 Sat 09:30-22:00 Sun 09:30-22:00
9am-11pm
2013
24小时
+8
10：00-24：00
Jan-Dec: Mo-Su 11:00-23:00
高碑店街道便民服务中心
Jan-Dec: Mo-Su 10:30-20:30
ALL
9:00am to 2:00am
10am - 11pm
10am - 9pm
Mon-Sun 10.00-24.00
18:00 - 22:00
Mo-Fr 06:00-10:00 am
24/24
24小时
24h
9:00 - 22:00
11:00~21:00
17-02
9:00~22:00
9:30~24:00
24h
10:00~22:00
9:30~21:30


然后再使用 `clean.process_opening_hours()` 函数处理 node_tag 数据中的时间格式。返回值是不能被统一的时间格式，它们被认为是不正确或者没有意义的营业时间信息，将被丢弃。

In [19]:
wrong_hour = clean.process_opening_hours(node_tag)
for h in wrong_hour:
    print(h)

2013
+8
高碑店街道便民服务中心
17-02


为了验证清洗的效果，我们再次运行 `clean.find_mess_hour()` 函数，其返回值是空，说明时间数据都已经被统一成规定格式了。

In [20]:
wrong_index, wrong_hour = clean.find_mess_hour(node_tag)
print(wrong_hour)

[]


同样，我们对 way_tag 数据中的营业时间也采用相同的处理方法。首先列出需要处理的数据。

In [21]:
wrong_index, wrong_hour = clean.find_mess_hour(way_tag)
for h in wrong_hour:
    print(h)

10am-10pm
11:00~12:00,17:00~19:00
举办日期: 2014年10月31日到11月20日  圆顶建筑开放时间: 09:30到20:30  室外放映时间: 19:00到21:00
不确定
08.30AM-05.30PM


经处理后，以下是不能被统一的数据。（下方的第一条数据虽然有完整的时间，但由于其格式过于特殊，没有可复用性，不值得单独编写相应的代码来处理。）

In [22]:
wrong_hour = clean.process_opening_hours(way_tag)
for h in wrong_hour:
    print(h)

举办日期 2014年10月31日到11月20日  圆顶建筑开放时间: 09:30到20:30  室外放映时间: 19:00到21:00
不确定


### 清洗门牌号码

首先，我们来查看若干条门牌号码的数据。顾名思义，门牌号需要包含数字才算正确，下方最后一条数据“奥北南区”显然不是门牌号，需要被清除出去。

In [23]:
node_tag[node_tag.key=='housenumber'].head()

,id,key,value,type
2435,292592805,housenumber,37,addr
4021,368122502,housenumber,30,addr
5520,600243400,housenumber,3号楼,addr
5569,600265656,housenumber,16,addr
6942,734834942,housenumber,奥北南区,addr


我们将不包含数字的门牌号码数据认为是错误的信息，并将他们从整体数据中删除。clean.py 清洗代码中的 `process_house_number()` 函数就是用于处理这类错误门牌号的。

In [24]:
def process_house_number(df):
    '''清洗数据中的门牌号码。
    如果housenumber字段中不包含数字，则认为是错误的门牌，会被丢弃。'''

    wrong_index = []
    wrong_list = []
    for index, row in df[df.key=='housenumber'].iterrows():
        house = row['value']
        if not re.search('\d', house):  # 判断是否包含数字
            wrong_index.append(index)
            wrong_list.append(house)
    
    df.drop(wrong_index, inplace=True)   # 删除数据中错误的门牌号码
    
    return wrong_list

我们使用 `clean.process_house_number()` 函数处理 node_tag、way_tag 数据中的门牌信息, 返回值是要被丢弃的错误数据。

In [25]:
wrong_list = clean.process_house_number(node_tag)
for w in wrong_list: 
    print(w)

奥北南区
北京市朝阳区
北京大学计算科学与技术系
八门
嘉园路
home guanganmen
北京中联华康科技有限公司


In [26]:
wrong_list = clean.process_house_number(way_tag)
for w in wrong_list: 
    print(w)

school
凯隆公寓
西院
西院
百子湾南路
鲁谷大街
Westin Chaoyang


### 补充缺失的 name 属性值

在地图数据中，tag 属性 key=name 是某一 id 下对应的名称，而 key=zh 属性是其对应的中文名称。但是我们发现有这样一种情况存在，在同一 id 下，存在 zh 属性，却不存在 name 属性，我们称这样的情况为 name 属性值的缺失。在这时，我们可以用 zh 属性的值来补充缺失的 name 属性值。

下面给出了一个缺失 name 属性值的例子。

In [27]:
groups = node_tag.groupby('id')
for iid, group in groups:
    if (not (group.key=='name').any()) and (group.key=='zh').any():
        print(group)
        break    

             id        key                    value     type
1482  274748336    amenity                fast_food  regular
1483  274748336    cuisine          american;burger  regular
1484  274748336         en               McDonald's     name
1485  274748336         zh                      麦当劳     name
1486  274748336    website  http://mcdonalds.com.cn  regular
1487  274748336     street                    甜水园餐厅     addr
1488  274748336  zh_pinyin               Màidāngláo     name


以下是 clean.py 清洗代码中进行 name 缺失属性值补充的函数 `process_name()`。

In [28]:
def process_name(df):
    '''当同一id下，存在zh属性却不存在name属性时， 增加name属性值，其值等同于zh的值。'''

    groups = df.groupby('id')   # 根据id将数据分组
    
    for iid, group in groups:
        if (not (group.key=='name').any()) and (group.key=='zh').any():
            name = group.loc[group.key=='zh', 'value']
            new_row = pd.DataFrame({'id': iid, 'key': 'name', 'value':name, 'type': 'regular'})
            df = df.append(new_row, ignore_index=True)
    
    return df

调用 `clean.process_name()` 函数分别对 node_tag 和 way_tag 数据中缺失的 name 属性值进行补充。

In [29]:
node_tag = clean.process_name(node_tag)

In [30]:
way_tag = clean.process_name(way_tag)

## 将清洗后的数据存储到SQL数据库

在数据清洗工作完成以后，我们需要将地图数据导入到SQL数据库中，方便后续的查询和分析。

首先，我们将上述清洗后的 DataFrame 格式数据分别存储到新的csv文件 nodes_tags_cleaned.csv 和 ways_tags_cleaned.csv 中。

In [31]:
node_tag.to_csv('nodes_tags_cleaned.csv', index=False)
way_tag.to_csv('ways_tags_cleaned.csv', index=False)

然后，我们将 csv 文件导入到 SQL 数据库中，这里使用的是 SQLite 数据库引擎。

以下代码创建名为 openstreet 的数据库，并定义了关于节点和途径信息的五张数据表。

In [32]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 

# 创建数据库
engine = create_engine("sqlite:///openstreet.sqlite") 
metadata = MetaData()
                       
# 定义数据库表

node_table = Table('nodes', metadata,
                   Column('index', Integer, primary_key=True, autoincrement=True),
                   Column('id', Integer),
                   Column('lat', Float),
                   Column('lon', Float),
                   Column('user', String),
                   Column('uid', Integer),
                   Column('version', String),
                   Column('changeset', Integer),
                   Column('timestamp', String)
                  )
                          
                       
nodetag_table = Table('nodes_tags', metadata,
                      Column('index', Integer, primary_key=True, autoincrement=True),
                      Column('id', Integer),
                      Column('key', String),
                      Column('value', String),
                      Column('type', String)
                     )

                          
way_table = Table('ways', metadata,
                   Column('index', Integer, primary_key=True, autoincrement=True),
                   Column('id', Integer),
                   Column('user', String),
                   Column('uid', Integer),
                   Column('version', String),
                   Column('changeset', Integer),
                   Column('timestamp', String)
                 )                     
                
waynode_table = Table('ways_nodes', metadata,
                      Column('index', Integer, primary_key=True, autoincrement=True),
                      Column('id', Integer),
                      Column('node_id', Integer),
                      Column('position', Integer)
                     )                          
                          
waytag_table = Table('ways_tags', metadata,
                     Column('index', Integer, primary_key=True, autoincrement=True),
                     Column('id', Integer),
                     Column('key', String),
                     Column('value', String),
                     Column('type', String)
                    )

                                      
# 创建数据表
metadata.create_all(engine)



下方定义的 `csv2db()` 函数，作用是将 csv 文件中的数据导入到数据库表中。

In [33]:
def csv2db(file, engine, table): 
    
    from sqlalchemy import insert
    
    data = pd.read_csv(file, dtype=str)
    data_list = list(data.apply(dict, axis=1))
    
    connection = engine.connect()
    stmt = insert(table)
    results = connection.execute(stmt, data_list)
    connection.close()


调用 `csv2db()` 函数，将5份关于节点和途径的csv文件数据导入到相应的数据表中。至此，地图数据导入数据库的工作完成。

In [34]:
csv2db('nodes.csv', engine, node_table)
csv2db('nodes_tags_cleaned.csv', engine, nodetag_table)
csv2db('ways.csv', engine, way_table)
csv2db('ways_nodes.csv', engine, waynode_table)
csv2db('ways_tags_cleaned.csv', engine, waytag_table)

## 数据概述

### 数据文件的大小
通过调用Linux命令，我们可以查看原始数据，中间过程的csv文件，以及新建的数据库。原始数据 beijing_china.osm 文件有175M，数据库 openstreet.sqlite 大小为96M。

In [35]:
!du -h *.osm *.sqlite *.csv

175M	beijing_china.osm
 18M	sample_beijing_china.osm
 96M	openstreet.sqlite
 65M	nodes.csv
2.9M	nodes_tags.csv
2.8M	nodes_tags_cleaned.csv
6.9M	ways.csv
 23M	ways_nodes.csv
8.0M	ways_tags.csv
7.8M	ways_tags_cleaned.csv


### 关于节点和途径的统计量

#### 节点（nodes）的数量

In [36]:
query = 'SELECT COUNT(*) FROM nodes'
pd.read_sql_query(query, engine)

,COUNT(*)
0,823210


####  旅馆的数量

考虑特殊的节点，比如旅馆，在nodes_tags表中需要满足 key='tourism'以及value='hotel'。

In [37]:
query = "SELECT COUNT(id) FROM nodes_tags WHERE key='tourism' AND value='hotel'"
pd.read_sql_query(query, engine)

,COUNT(id)
0,343


####  途径（ways）的数量

In [38]:
query = 'SELECT COUNT(*) FROM ways'
pd.read_sql_query(query, engine)

,COUNT(*)
0,122904


####  高架桥的数量

考虑特殊的途径，比如高架桥，在wyas_tags表中需要满足条件key='bridge'和value='viaduct'。

In [39]:
query = "SELECT count(id) FROM ways_tags WHERE key='bridge' AND value='viaduct'"
pd.read_sql_query(query, engine)

,count(id)
0,165


### 关于用户的统计量

#### 唯一用户的数量

这里需要同时统计nodes和ways这两张表。

In [40]:
query = '''SELECT COUNT(DISTINCT uid) 
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)'''
pd.read_sql_query(query, engine)

,COUNT(DISTINCT uid)
0,1720


#### 贡献数据量前十的用户

In [41]:
query = '''SELECT user, COUNT(user) AS num 
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) 
GROUP BY user 
ORDER BY num DESC 
LIMIT 10'''
pd.read_sql_query(query, engine)

,user,num
0,Chen Jia,213003
1,R438,143481
2,hanchao,66897
3,ij_,51905
4,Алекс Мок,47725
5,katpatuka,23621
6,m17design,21699
7,Esperanza36,18475
8,nuklearerWintersturm,16502
9,RationalTangle,13768


### 信号灯最多的途径

利用ways_nodes表可以将途径和节点关联起来，这样就可以知道每条途径上的节点情况，再联合查询nodes_tags表，就能筛选出每条途径上的特殊节点。这里将ways_nodes和nodes_tags这两张表通过ways_nodes.node_id=nodes_tags.id关联起来。

下面查询的是信号灯最多的前10条途径。

In [42]:
query = '''SELECT ways_nodes.id, COUNT(ways_nodes.id) AS sig_num 
FROM ways_nodes JOIN nodes_tags ON ways_nodes.node_id=nodes_tags.id
WHERE nodes_tags.value='traffic_signals'
GROUP BY ways_nodes.id
ORDER BY sig_num DESC
LIMIT 10'''
pd.read_sql_query(query, engine)

,id,sig_num
0,251324796,23
1,237581600,22
2,145291213,21
3,145291215,21
4,159791637,21
5,127156384,19
6,187044296,18
7,187044304,18
8,31757284,17
9,154855573,17


以上给出的是途径id和该途径上的信号灯数量，如果想要同时知道该途径的具体名称，则还要联合ways_tags表来查询。这里将上面的查询结果作为一个新的表e，和ways_tags表进行联合查询。

In [43]:
query = '''SELECT ways_tags.id, ways_tags.value, e.sig_num
FROM ways_tags JOIN 
(SELECT ways_nodes.id, COUNT(ways_nodes.id) AS sig_num 
FROM ways_nodes JOIN nodes_tags ON ways_nodes.node_id=nodes_tags.id
WHERE nodes_tags.value='traffic_signals'
GROUP BY ways_nodes.id
ORDER BY sig_num DESC
LIMIT 10) e
ON ways_tags.id=e.id
WHERE ways_tags.key='name'
ORDER BY e.sig_num DESC
'''
pd.read_sql_query(query, engine)

,id,value,sig_num
0,251324796,北土城东路,23
1,237581600,北土城东路,22
2,145291213,大屯路,21
3,145291215,大屯路,21
4,159791637,北土城东路,21
5,127156384,朝阳路,19
6,187044296,朝阳路,18
7,187044304,朝阳路,18
8,31757284,朝阳路,17
9,154855573,石景山路,17


我们注意到结果中不同id的途径具有相同的名称（value）值，这大概是由于一条道路需要由多条途径（ways）组成。

## 关于数据集的其他想法

### 数据标注的完整性讨论

我们以节点（nodes）数据为例，通过下面的查询发现，总的节点数为823210个，而被标记（也就是具有tag属性）的节点数据只有40812个，占了不到总数的5%。可见，目前北京地区的地图信息还是不够全面丰富的，有待更进一步的补充和完善。

完善地图数据的标注信息是一个非常有挑战性的问题，但同时也会提供更丰富的数据分析维度。如何增加数据的标注信息，这里给出若干个想法：
* 由于OpenStreetMap是一个开放的可由大家共同编辑的地图数据集，所以需要增加大家贡献数据的积极性，让更多的人愿意参与进来，比如可以通过节点的重要性来设置奖励等级等一系列互动好玩的方法增强积极性。
* 除了由人力众筹的方式贡献数据，还可以采用机器，从网络或其他途径采集相关的数据。
* 当然，在增加标注信息的时候，也要注意数据的准确性，数据中可以增加数据可信度等级，当有更多人对这一信息认可时它的可信度等级就越高。

In [44]:
# 总节点数
query = 'SELECT COUNT(DISTINCT id) FROM nodes'
nodes_num = pd.read_sql_query(query, engine).iloc[0,0]
nodes_num

823210

In [45]:
# 被标注的节点数
query = 'SELECT COUNT(DISTINCT id) FROM nodes_tags'
taged_nodes_num = pd.read_sql_query(query, engine).iloc[0,0]
taged_nodes_num

40812

In [46]:
# 标注节点数占总节点数的比例
taged_nodes_num / nodes_num

0.049576657232055003

### 数据地理分布情况的讨论。

这里我们来讨论节点数据在北京范围内的分布情况。首先我们需要找到北京中心的经纬度，并以此为中心将北京划分为东北、西北、东南、西南四个区域。然后查看每个区域内的节点数量。

以下的查询是首先找到北京市的id，并通过该id查找北京所在的经纬度，并认为这就是北京中心地点的坐标。

In [47]:
# 查找北京市的id信息
query = "SELECT * FROM nodes_tags WHERE key='name' AND value='北京市'"
pd.read_sql_query(query, engine)

,index,id,key,value,type
0,2,25248662,name,北京市,regular


In [48]:
# 查找北京市的经纬度信息
query = "SELECT * FROM nodes WHERE id=25248662"
pd.read_sql_query(query, engine)

,index,id,lat,lon,user,uid,version,changeset,timestamp
0,1,25248662,39.905963,116.391248,ff5722,3450290,87,44008181,2016-11-28T14:28:46Z


有了北京中心的坐标后，就可以根据每个id携带的经纬度数据，统计在四个不同区域内的节点数据了，并比较每个区域节点数占总节点数比例的情况。

* 在东北方向，总共有225099个节点，占总节点数的27.34%。

In [49]:
# 东北方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat>39.905963 AND lon>116.391248"
east_north = pd.read_sql_query(query, engine).iloc[0,0]
east_north

225099

In [50]:
# 东北方向节点数占比
east_north / nodes_num

0.27344055587274207

* 在西北方向，总共有291253个节点，占总节点数的35.38%。

In [51]:
# 西北方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat>39.905963 AND lon<116.391248"
west_north = pd.read_sql_query(query, engine).iloc[0,0]
west_north

291253

In [52]:
# 西北方向节点数占比
west_north / nodes_num

0.35380158161344005

* 在东南方向，总共有175337个节点，占总节点数的21.30%。

In [53]:
# 东南方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat<39.905963 AND lon>116.391248"
east_south = pd.read_sql_query(query, engine).iloc[0,0]
east_south

175337

In [54]:
# 东南方向节点数占比
east_south / nodes_num

0.21299182468628905

* 在西南方向，总共有131520个节点，占总节点数的15.98%。

In [55]:
# 西南方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat<39.905963 AND lon<116.391248"
west_south = pd.read_sql_query(query, engine).iloc[0,0]
west_south

131520

In [56]:
# 西南方向节点数占比
west_south / nodes_num

0.15976482307066242

从上面计算的数据中，我们发现处在东北、西北方向的节点是远多于东南、西南方向的节点数的。北京北部地区的节点数占了62.72%，而南部地区只占了37.28%。这大概与北京南北区域发展不平衡有关，一般来说，北京的北边的地区经济更为发达，人口更为密集，而南边地区相对来说发展的没那么好，所以导致了节点数据在地理位置上分布的不均衡。

In [57]:
# 北部地区节点占比
(east_north + west_north) / nodes_num

0.62724213748618218

In [58]:
# 南部地区节点占比
(east_south + west_south) / nodes_num

0.37275664775695144

以上关于数据在地理分布的上的讨论还是比较粗浅的，只是粗暴的把北京地区分为四个大致的区域。其实还可以根据具体的城区（比如海淀区、朝阳区等）来做进一步的细分，但是目前缺少关于节点所在城区的细节信息，这也是现有的数据中可以加以完善的方面。如果有了更进一步的城区信息，就可以对不同城区的数据做更精细的分析，比如分析各城区的商店、银行、道路情况等信息，从而观察它们的经济发展水平。

## 总结

这篇报告整理并分析了OpenStreetMap开放地图中关于北京地区的数据。
* 首先，是对数据做了一些清洗工作，比如对电话号码、营业时间格式做了统一，剔除了错误的邮政编码和门牌号数据，以及补全了缺失的name属性值。

* 然后，将清洗后的数据存入SQLite数据库中，并通过SQL查询语句统计了数据中的若干信息，比如节点和途径的相关统计、用户的相关统计，以及查看了信号灯最多的前十条途径和它们的名称。

* 最后，给出了关于数据集的一些其他想法，这里讨论了数据标注的完整性问题，以及节点在地理分布上的差异情况。

## 参考资源

* [OpenStreetMap官网](https://www.openstreetmap.org/)
* [OpenStreetMap 维基页面](https://wiki.openstreetmap.org/wiki/Main_Page)
* [Mapzen 北京地区数据下载地址](https://mapzen.com/data/metro-extracts/metro/beijing_china/)
* [手机号码百度百科](https://baike.baidu.com/item/手机号码)
* [北京邮编区号大全](http://www.ip138.com/post/beijing/)